# RNN Encoder-Decoder

This is the notebook for constructing Theano RNN layers and RNN sentiment classifier. The text for an encoder-decoder can be found in a lot of places, and one reference is here: [http://anie.me/rnn-encoder-decoder/](http://anie.me/rnn-encoder-decoder/). This notebook might also contain a section on vanilla RNN classification task, and a section on RNN autoencoder as well. Note in this implementation, we are being a lot more efficient than some blog posts (like in WildML).

In [2]:
from classifier.rnn_layers import *
from classifier.util import *
from data.snli_util import *
from vis.util import *
import numpy as np

# import matplotlib.pyplot as plt

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data SNLI

We use SNLI for inference task (the main purpose of encoder-decoder).

In [3]:
data = load_data('data/snli_data.npz')

# NOTE: right now this doesn't work, but we want to hold it to 
# the same standard as other code
# Print out all the keys and values from the data dictionary
for k, v in data.iteritems():
  if type(v) == np.ndarray:
    print k, type(v), v.shape, v.dtype
  else:
    print k, type(v), len(v)

loading training sentences...
loading dev sentences...
loading test sentences...
train_sentences <type 'numpy.ndarray'> (549367,) object
y_dev <type 'numpy.ndarray'> (9842,) int32
y_train <type 'numpy.ndarray'> (549367,) int32
W_embed <type 'numpy.ndarray'> (34047, 300) float64
idx_word_map <type 'numpy.ndarray'> (34047,) <U20
dev_sentences <type 'numpy.ndarray'> (9842,) object
test_sentences <type 'numpy.ndarray'> (9824,) object
y_test <type 'numpy.ndarray'> (9824,) int32
word_idx_map <type 'numpy.ndarray'> () object


## Inner Attention Layer

Inner attention layer is a single layer that's on top of LSTM/RNN. This layer chanegs/recomputes the hidden state. This is from the paper [LSTMN on Machine Reading](http://arxiv.org/pdf/1601.06733v3.pdf). Inner attention layer can be implemented on both encoder and decoder.

## Global Attention Layer

Global attention layer is very traditional, reference [Luong's paper](http://arxiv.org/pdf/1508.04025v5.pdf). They also implemented a local attention layer (which is very different from Inner attention layer, and more similar to hard vs. soft attention). We skip local attention layer in this build.